# Libraries

In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

import tensorflow as tf
import zipfile
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
#import tensorflow as tf
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Add


from keras import models
from keras import optimizers
from keras.preprocessing import image
from keras.callbacks import CSVLogger

import numpy as np
import pandas as pd


In [2]:
print(tf.keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


### Necesary versions of Keras and Tensorflow

In [3]:
!pip install keras==2.4.3

In [4]:
!pip3 install tensorflow==2.5.0 --user

In [5]:
# Function to create I3D model
from i3d_merged_model import Inception_Inflated3d

### Below some things that were needed to use GPU

In [6]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.95
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [8]:
#GPU did not use 
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"], 
#                                         cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

# Step 1: Changes to ID3 Inflated inception model.py
- I delete all the names for conv and batch normalization (one model could not have repeated names)
- I added the functions to get some channels of the input layer of 5 channels(video) to process the model with rgb and flow at the same time

# Step 2: Look the tensor channels

### NOT RUN
Only to try if they could work with ID3. The final functions are inside the ID3 model.

In [9]:
# extract the rgb images (the same as RGB model are in space 3 (greyscale))
# from keras.layers import Lambda
# #from keras.backend import slice

# def get_rgb(input_x):
#     rgb = input_x[...,:3]
#     return rgb

# # extract the optical flows (they are 2d vectors, the final 2)
# def get_opt(input_x):
#     opt= input_x[...,3:5]
#     return opt

In [10]:
# inputs = Input(shape=(None,224,224,5))

# #Wraps arbitrary expressions as a Layer object.
# rgb = Lambda(get_rgb,output_shape=None)(inputs)
# opt = Lambda(get_opt,output_shape=None)(inputs)

In [11]:
#opt

In [12]:
#inputs

In [13]:
#rgb

In [14]:
#look if the rgb output of the lambda is a real tensor to use in id3 model
#K.is_keras_tensor(rgb)

# Step 3: Build the model 

In [15]:
inputs = Input(shape=(None,224,224,5))

### RGB

In [16]:
# Parameters for the model
NUM_FRAMES = 79
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_CLASSES = 2

# Parameters
dropout_prob = 0.0

In [17]:
# Create the I3D model without top layer and using pretrained weights
rgb_model = Inception_Inflated3d(
                include_top=False,
                weights='rgb_imagenet_and_kinetics',
                input_tensor=inputs,
                input_name=1,
                input_shape=(None, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS), 
                classes=NUM_CLASSES)

In [18]:
# Creating sequential model
rgb = rgb_model.output 
rgb = tf.reshape(rgb,[-1,64, 14, 14, 32]) #this shape has the dimension of rwf2000 before adding rgb and flow

In [19]:
rgb

<KerasTensor: shape=(None, 64, 14, 14, 32) dtype=float32 (created by layer 'tf.reshape')>

In [20]:
# for layer in rgb_model.layers:
#     print(layer, layer.trainable)

In [21]:
# Make only the last two inception modules and top layer trainable (MAYBE CHANGE THE 21 TO LESS)
for layer in rgb_model.layers[:-21]: #aca fue el ultimo nomas fuera (le sacaria 42)
    layer.trainable = False

### Flow

In [22]:
# Parameters for the model
NUM_FRAMES = 79
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_FLOW_CHANNELS = 2
NUM_CLASSES = 2

In [23]:
# Create the I3D model without top layer and using pretrained weights
flow_model = Inception_Inflated3d(
                include_top=False,
                weights='flow_imagenet_and_kinetics',
                input_tensor = inputs,
                input_name = 2,
                input_shape=(None, FRAME_HEIGHT, FRAME_WIDTH, NUM_FLOW_CHANNELS),
                classes=NUM_CLASSES)

C:\Users\jdela\OneDrive\Desktop\DL\at3\violence_detection\I3D\i3d_merged_model.py:98: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


In [24]:
# Creating sequential model
opt = flow_model.output
opt = tf.reshape(opt,[-1,64, 14, 14, 32]) #same shape as rfw2000 before merging block


In [25]:
# Make only the last two inception modules and top layer trainable
for layer in flow_model.layers[:-21]: #change to be 42  
    layer.trainable = False

In [26]:
opt

<KerasTensor: shape=(None, 64, 14, 14, 32) dtype=float32 (created by layer 'tf.reshape_1')>

### Fusion like RWF2000

In [27]:
##################################################### Fusion and Pooling
x = Add()([rgb,opt])
x = MaxPooling3D(pool_size=(8,1,1))(x)

# # # ##################################################### Merging Block
x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

x = Conv3D(
     64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
     64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

x = Conv3D(
     128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
     128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,3,3))(x)

# # # ##################################################### FC Layers
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

# # # Build the model
pred = Dense(2, activation='softmax')(x)
newModel = Model(inputs, pred) #inputs are the layer with the 5 channels


In [28]:
newModel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 224, 2 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, None, 224, 22 0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 224, 22 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, None, 112, 11 65856       lambda[0][0]                     
______________________________________________________________________________________________

In [29]:
for layer in newModel.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001C74C94F2C8> False
<keras.layers.core.Lambda object at 0x000001C76435B588> False
<keras.layers.core.Lambda object at 0x000001C7644789C8> False
<keras.layers.convolutional.Conv3D object at 0x000001C75DE6DE88> False
<keras.layers.convolutional.Conv3D object at 0x000001C76489B348> False
<keras.layers.normalization_v2.BatchNormalization object at 0x000001C76436E9C8> False
<keras.layers.normalization_v2.BatchNormalization object at 0x000001C7646F2308> False
<keras.layers.core.Activation object at 0x000001C7643BB0C8> False
<keras.layers.core.Activation object at 0x000001C76489B508> False
<keras.layers.pooling.MaxPooling3D object at 0x000001C7644033C8> False
<keras.layers.pooling.MaxPooling3D object at 0x000001C7646FAE88> False
<keras.layers.convolutional.Conv3D object at 0x000001C764408B48> False
<keras.layers.convolutional.Conv3D object at 0x000001C7646FF588> False
<keras.layers.normalization_v2.BatchNormalization object at 0x000001C7644

# Step 4: Create data generators

In [ ]:
# Import custom data generator class
from i3ddatagenerator import DataGenerator

In [ ]:
# Parameters
batch_size = 8
path_train = '../datai3d/rgb/train/'
path_val = '../datai3d/rgb/validation/'
path_test = '../datai3d/rgb/test/'

In [ ]:
# Create data generators for training, validation and test
train_generator = DataGenerator(directory=path_train,
                                batch_size=batch_size, 
                                data_augmentation=True)

validation_generator = DataGenerator(directory=path_val,
                                     batch_size=batch_size,
                                     data_augmentation=False,
                                     target_frames = None,
                                     flip = False)

test_generator = DataGenerator(directory=path_test,
                               shuffle = False,
                               batch_size=batch_size, 
                               data_augmentation=False,
                               target_frames = None,
                               flip = False)

# Step 5: Train RGB model

In [ ]:
from keras import optimizers

In [ ]:
# Parameters
epochs = 100
steps_per_epoch = train_generator.n_files//batch_size
validation_steps = validation_generator.n_files//batch_size

# Path to store checkpoints
filepath = 'checkpoints/weights_i3drgb.hdf5'

In [ ]:
# Callbacks

# Reduce learning rate by 10x when plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.1,
                                                 patience=5,
                                                 min_lr=0.0001,
                                                verbose = 1)

# Save best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                monitor='val_loss', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                save_weights_only=False, 
                                                mode='auto', 
                                                save_freq='epoch')

# Stop after 10 epochs if val loss does not improve
earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                             restore_best_weights = True, 
                                             patience = 10, 
                                             min_delta = 0.01,
                                            verbose = 1)

callbacks = [reduce_lr, checkpoint, earlystop]

In [ ]:
# Create optimiser (SGD with momentum)
optimizer = optimizers.SGD(learning_rate = 0.01, momentum = 0.9)

# Compile model
model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
# Start training and save history
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks = callbacks,
    verbose=1)

In [ ]:
# Save history of the model
import pickle
with open('history_i3drgb', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Step 6: Test model

In [ ]:
# Parameters
test_steps = test_generator.n_files//batch_size

In [ ]:
# Evaluate model on test set
model.evaluate(test_generator, 
               steps = test_steps, 
               return_dict =  True)